### 접근
- 검은 점과 흰 점으로 볼록껍질을 각각 구한 뒤, 그 직선끼리 닿는지 여부를 파악한다.
- 닿지 않는다면 분리할 수 있는 것일 것이다.
- 꼭짓점의 개수 n+m개에 대해서 $O((n+m)^2)$만큼 걸릴 것 같다.
  - 그런데 어떤 볼록껍질이 다른 볼록껍질을 완전히 포함하는 경우가 있는데, 이 경우를 처리할 수 없다..

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

ccw = lambda o, a, b: (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
def cross(a, b) :
  t1 = ccw(a[0], a[1], b[0]) * ccw(a[0], a[1], b[1])
  t2 = ccw(b[0], b[1], a[0]) * ccw(b[0], b[1], a[1])
  if t1 < 0 and t2 < 0 : return True
  if t1 == 0 and t2 == 0: return b[0] <= a[1] and a[0] <= b[1]
  return t1 <= 0 and t2 <= 0

def andrews(P):
  P = sorted(P)

  lo = []
  for p in P:
    while len(lo) >= 2 and ccw(lo[-2], lo[-1], p) <= 0:
      lo.pop()
    lo.append(p)
  lo.pop()

  up = []
  for p in reversed(P) :
    while len(up) >= 2 and ccw(up[-2], up[-1], p) <= 0:
      up.pop()
    up.append(p)
  up.pop()
  
  return lo + up

def sol() :
  N, M = map(int, input().split())
  H1 = andrews([tuple(map(int, input().split())) for _ in range(N)])
  H2 = andrews([tuple(map(int, input().split())) for _ in range(M)])
  

for _ in range(int(input())) :
  sol()

- 볼록다각형 내부 점 판정문제부터 풀고 다시 재개

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

ccw = lambda o, a, b: (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
def inside(L, s) :
  if ccw(L[0], L[1], s) < 0 : return False
  if ccw(L[0], L[-1], s) > 0 : return False

  l, r = 1, len(L) - 1
  while l + 1 < r :
    mid = (l + r) // 2
    if ccw(L[0], L[mid], s) > 0 : l = mid
    else : r = mid

  A, B = l, l+1
  return ccw(L[A], s, L[B]) < 0

def cross(a, b) :
  t1 = ccw(a[0], a[1], b[0]) * ccw(a[0], a[1], b[1])
  t2 = ccw(b[0], b[1], a[0]) * ccw(b[0], b[1], a[1])
  if t1 < 0 and t2 < 0 : return True
  if t1 == 0 and t2 == 0: return b[0] <= a[1] and a[0] <= b[1]
  return t1 <= 0 and t2 <= 0

def andrews(P):
  if len(P) <= 2 : return P
  P = sorted(P)

  lo = []
  for p in P:
    while len(lo) >= 2 and ccw(lo[-2], lo[-1], p) <= 0:
      lo.pop()
    lo.append(p)
  lo.pop()

  up = []
  for p in reversed(P) :
    while len(up) >= 2 and ccw(up[-2], up[-1], p) <= 0:
      up.pop()
    up.append(p)
  up.pop()
  
  return lo + up

def sol() :
  N, M = map(int, input().split())
  if not N or not M : return True

  H1 = andrews([tuple(map(int, input().split())) for _ in range(N)])
  H2 = andrews([tuple(map(int, input().split())) for _ in range(M)])

  if len(H2) > 2: #모든 점은 다른 도형의 내부에 있으면 안됨
    for h1 in H1 : 
      if inside(H2, h1) : return False
  
  if len(H1) > 2:
    for h2 in H2 :
      if inside(H1, h2) : return False
  
  if len(H1) == 1 : H1.append(H1[0]) #점이 하나만 있을 경우 선분처럼 처리할 수 있게 만든다.
  if len(H2) == 1 : H2.append(H2[0])
  
  for i, p1 in enumerate(H1) : #각 선분이 교차하는지 확인한다.
    for j, p3 in enumerate(H2) :
      p2 = H1[(i+1) % len(H1)]
      p4 = H2[(j+1) % len(H2)]
      if cross((p1, p2), (p3, p4)) : return False 

  return True

for _ in range(int(input())) :
  if sol() : sys.stdout.write("YES\n")
  else : sys.stdout.write("NO\n")

### 풀이
- 우선 각 n, m개의 점으로 볼록껍질을 구한다.
- 모든 점이 다른 볼록껍질 내부에 포함되어있지 않고, 모든 선분이 다른 볼록껍질을 이루는 선분과 닿지 않으면 분리할 수 있다.
  - 이때 볼록껍질의 내부에 포함되는지 확인하기 위해선 그 볼록껍질의 점의 개수가 3개 이상이여야 한다.
  - 선분 교차판정을 확인하려면 Point가 4개가 필요하다.
    - 이 부분은 n이나 m이 1일 때 그 뒤에 똑같은 원소를 추가하는 식으로 해결했다.
    - ex) H1 = [(10, 10), (10, 10)]

### 오답노트
- indexError가 여러개 났다. 상기한 볼록껍질 내부에 있는지 확인하는 과정에서 `len(H2) > 2`면 `inside(H2, h1)`로 했어야 했는데 `H1`을 확인하다가 문제가 계속 생겼었다.